In [7]:
import numpy as np
np.repeat(np.array([[2,3]]), 4,axis=0)

array([[2, 3],
       [2, 3],
       [2, 3],
       [2, 3]])

In [ ]:
path = '..\data\iScat\Data\\2024_11_11\\Metasurface\\Chip_02\\Metasurface 01\\Brightfield'
z_stack_path = path+'\\02_01_BF.nd2'
f1_path = path+'\\Captured Cy5.tif'
f2_path = path+'\\Captured FITC.tif'
f3_path = path+'\\Captured TRITC.tif'
f1_results = path+"\\Cy5.csv"
f2_results = path+"\\FITC.csv"
f3_results = path+"\\TRITC.csv"

In [9]:
import os
a=os.path.join('data', 'iScat','Data','2024_11_11','Metasurface','Chip_02')

In [ ]:
import os

def get_data_paths(root_path,mode="Brightfield"):
    nd2_files = []
    tiff_files = []
    
    for dirpath, dirnames, filenames in os.walk(root_path):
        # Focus only on directories named 'Brightfield'
        if os.path.basename(dirpath) == mode:
            for file in filenames:
                if file.endswith('.nd2'):  # Check for .nd2 files
                    nd2_path = os.path.join(dirpath, file)
                    # Paths for the TIFF files
                    cy5_path = os.path.join(dirpath, 'Captured Cy5.tif')
                    fitc_path = os.path.join(dirpath, 'Captured FITC.tif')
                    tritc_path = os.path.join(dirpath, 'Captured TRITC.tif')
                    
                    # Ensure all three TIFF files exist
                    if all(os.path.exists(path) for path in [cy5_path, fitc_path, tritc_path]):
                        nd2_files.append(nd2_path)
                        tiff_files.append((cy5_path, fitc_path, tritc_path))
    
    return nd2_files, tiff_files

# Example usage
root_path = os.path.join('..','data', 'iScat', 'Data', '2024_11_11', 'Metasurface', 'Chip_02')
nd2_list, tiff_tuples = get_data_paths(root_path)